In [ ]:
# requests와 gradio API를 Import함
import requests  
import gradio as gr

# open AI request 받아오는 함수
def request_gpt(prompt, histories=[]):
    ai_search_endpoint = "https://team9-ai-search.search.windows.net/"
    ai_search_key = "xVtlSUATNV7DwdHB4JOdzhLvYyEZz4LdUFgKXyWzJiAzSeAvOyxo"
    ai_search_index = "jeju-dialect-index-delete"
    endpoint = "https://team9-openai-ncus.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-type": "application/json",
        "api-key": "7U7x0fAWIHKby0AXBpGGr3zjI52iTr9xb7pFKmCH7eBnrdytwXrCJQQJ99BAACHrzpqXJ3w3AAABACOGuqAv"
    }
    
    message_list = list()
    
    # 시스템 메시지, 프롬프트 입력
    message_list.append({
        "role": "system",
        "content": """다음은 제주도 방언을 표준 한국어로 번역하고, 해당 방언의 단어가 가진 다양한 의미를 문맥에 따라 분석하는 역할입니다. 번역된 내용에 대해 참조한 문서도 함께 제공합니다. 출력 형식은 아래를 따릅니다.
 
# 출력 형식
1. **단어 및 문장 분석**: 주어진 방언 문장에 포함된 단어가 가진 다양한 의미를 나열합니다. 문맥에 따라 적합한 의미를 제공하며, 상황별로 해석할 수 있는 경우 이를 명확히 표기합니다.
2. **참조**: 문서나 데이터베이스를 기반으로 분석한 내용을 표시합니다. 해당 참조는 번호로 구분됩니다.
 
# 예제 출력
입력: "혼저 옵서예"
 
출력:
---
"혼저 옵서예"라는 문장에서 방언 단어들이 가진 다양한 의미는 다음과 같습니다:
1. 혼저:
   - 먼저
   - 어서
2. 옵서예:
   - 오세요
   - 방문해 주십시오
 
문맥에 따라 '혼저'는 "먼저" 또는 "어서"로 해석될 수 있으며, '옵서예'는 "오세요"로 번역됩니다.

 
# 지침
- 모든 단어를 가능한 한 자세히 분석합니다.
- 참조는 데이터베이스에서 검색한 결과에 기반하여 번호와 함께 출력합니다.
- 방언 해석이 모호하거나 다의적인 경우, 가능한 모든 의미를 제공합니다.
- 단어 및 문장의 최종 해석을 사용자에게 명확히 전달합니다.
- 참조는 결과 하단에 별도 섹션으로 제공합니다.
 
입력:
"{방언 문장}"


사용자가 입력한 제주도 방언 문장을 표준어로 번역합니다. 주어진 데이터에 포함된 방언 단어를 참조하여 표준어 문장을 반환하세요.
 
예시:
입력: '누게 있수과' 
출력: '누가 있습니까?'
 
입력: '겅 하니까이'
출력:  '그러니까'
입력: '예초하다가이 뱀 나온거야' 
출력: '예초하다가 뱀이 나온 거야'
제출된 문장에서 제주도 방언 단어를 찾아 주어진 데이터와 대조한 후, 가장 적절한 표준어 번역을 제공하세요.
"""       
        
    })

    # 이전 메시지 포함.
    for history in histories:
        for text in history:
            message_list.append({
                "role": "assistant",
                "content": text
                
            })

    # 유저 메시지.
    message_list.append({
        "role": "user",
        "content": prompt
    })            
    
    payload = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "semantic_configuration": "jeju-dialect-sementic-delete",
                "query_type": "semantic",
                "queryLanguage": "ko-kr",
                "strictness": 2,
                "top_n_documents": 10,
                "key": ai_search_key,
                "indexName": ai_search_index
            }
        }
        ],
    }  
    
    # endpoint와 headers, json에 payload requests함
    response = requests.post(endpoint, headers=headers, json=payload)
    print(response.status_code)
    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
        content = response_json["choices"][0]["message"]["content"]

        citations = response_json["choices"][0]["message"].get("context", {}).get("citations", [])
        formatted_citation_list = []
        
        if citations:
            for i, citation in enumerate(citations, start=1):
                formatted_citation = f"<details><summary>참조 {i}</summary><ul>{citation['content']}</ul></details>"
                formatted_citation_list.append(formatted_citation)

        return content, "".join(formatted_citation_list)
    else:
        return f"{response.status_code}, {response.text}", ""

#REST API를 통하여 STT FAST request
def request_stt_fast(file_path):
    endpoint = "https://eastus.api.cognitive.microsoft.com/speechtotext/transcriptions:transcribe?api-version=2024-11-15"
    headers = {
        "Ocp-Apim-Subscription-Key": "EXdCQatIT6OYg5RRAwYr7A6JTvRyDfYzJ8VS6HUMIUpKeB8wcKOqJQQJ99BAACYeBjFXJ3w3AAAYACOGWEOf",
    }

    with open(file_path, "rb") as audio :
        audio_data = audio.read()

    json = {
        "definition": '{"locales":["ko-KR"], "profanityFilterMode":"Masked", "channels":[0,1]}'
    }

    files = {
        "audio" : audio_data
    }

    response = requests.post(endpoint, headers=headers,files=files, json=json)

    print("Fast", response.status_code, response.elapsed.total_seconds())
    if response.status_code == 200:
        response_json = response.json()
        response_text = response_json['combinedPhrases'][0]['text']
        print(response_text)
        return response_text
    else:
        return ''

#전송 버튼을 눌렀을 시 함수
def click_send(prompt, histories):
    #response_text에 gpt 대답, citaion_html에 주석을 받음
    response_text, citation_html = request_gpt(prompt=prompt, histories=histories)
    histories.append((prompt, response_text))
    #리턴 값으로 gpt대답이 들어간 histories 리스트와 주석이 들어간 citation_html을 줌
    return histories, citation_html, ""

#오디오를 텍스트로 바꿔줌줌
def change_audio(file_path):
    #만약 file_path가 유효값 즉 음성 파일이 있으면
    if file_path:
        #response_text에 음성을 텍스트로 변환해줌
        response_text = request_stt_fast(file_path=file_path)
        return response_text
    else:
        #음성파일이 없으면 리턴값으로 빈값을 줌
        return ''



d:\utility\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with gr.Blocks() as demo:
    
    with gr.Tab(label="🗣️🔄제주방언 번역")  as tab2:
        gr.Markdown("🏝️제주방언 번역")

        # 음성 입력 및 결과 출력
        with gr.Column():
            input_mic = gr.Audio(
                label="🎤 마이크 입력",
                sources="microphone",
                type="filepath",
                interactive=True, waveform_options=gr.WaveformOptions(
                    waveform_color="#D3B1C2",
                    waveform_progress_color="#C197D@",
                    skip_length=2,
                    show_controls=False)
            )



        # 프롬프트 입력과 결과
        gr.Markdown("### 🗣️🔄 번역 내용")
        chatbot_gpt = gr.Chatbot(
            label="🗣️🔄 번역 내용", 
            height=200
        )
        #주석
        citation = gr.HTML(label="🔗 참조 링크")


        input_openai_textbox = gr.Textbox(
                label="📝 텍스트", 
                interactive=True, 
            )
        send_button = gr.Button("전송")
        
        input_mic.change(
            fn=change_audio, 
            inputs=[input_mic], 
            outputs=[input_openai_textbox]
        )
        send_button.click(
            fn=click_send,
            inputs=[input_openai_textbox, chatbot_gpt],
            outputs=[chatbot_gpt, citation, input_openai_textbox]
        )
        input_openai_textbox.submit(
            fn=click_send,
            inputs=[input_openai_textbox, chatbot_gpt],
            outputs=[chatbot_gpt, citation, input_openai_textbox]
        )

demo.launch()

d:\utility\Python\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [9]:
import requests 
import gradio as gr

# open AI request 받아오는 함수
def request_gpt(prompt, histories=[]):
    ai_search_endpoint = "https://team9-ai-search.search.windows.net/"
    ai_search_key = "xVtlSUATNV7DwdHB4JOdzhLvYyEZz4LdUFgKXyWzJiAzSeAvOyxo"
    ai_search_index = "jeju-dialect-index-delete"
    endpoint = "https://team9-openai-ncus.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-type": "application/json",
        "api-key": "7U7x0fAWIHKby0AXBpGGr3zjI52iTr9xb7pFKmCH7eBnrdytwXrCJQQJ99BAACHrzpqXJ3w3AAABACOGuqAv"
    }
    
    message_list = list()
    
    # 시스템 메시지
    message_list.append({
        "role": "system",
        "content": """다음은 제주도 방언을 표준 한국어로 번역하고, 해당 방언의 단어가 가진 다양한 의미를 문맥에 따라 분석하는 역할입니다. 번역된 내용에 대해 참조한 문서도 함께 제공합니다. 출력 형식은 아래를 따릅니다.
 
# 출력 형식
1. **단어 및 문장 분석**: 주어진 방언 문장에 포함된 단어가 가진 다양한 의미를 나열합니다. 문맥에 따라 적합한 의미를 제공하며, 상황별로 해석할 수 있는 경우 이를 명확히 표기합니다.
2. **참조**: 문서나 데이터베이스를 기반으로 분석한 내용을 표시합니다. 해당 참조는 번호로 구분됩니다.
 
# 예제 출력
입력: "혼저 옵서예"
 
출력:
---
"혼저 옵서예"라는 문장에서 방언 단어들이 가진 다양한 의미는 다음과 같습니다:
1. 혼저:
   - 먼저
   - 어서
2. 옵서예:
   - 오세요
   - 방문해 주십시오
 
문맥에 따라 '혼저'는 "먼저" 또는 "어서"로 해석될 수 있으며, '옵서예'는 "오세요"로 번역됩니다.

 
# 지침
- 모든 단어를 가능한 한 자세히 분석합니다.
- 참조는 데이터베이스에서 검색한 결과에 기반하여 번호와 함께 출력합니다.
- 방언 해석이 모호하거나 다의적인 경우, 가능한 모든 의미를 제공합니다.
- 단어 및 문장의 최종 해석을 사용자에게 명확히 전달합니다.
- 참조는 결과 하단에 별도 섹션으로 제공합니다.
 
입력:
"{방언 문장}"


사용자가 입력한 제주도 방언 문장을 표준어로 번역합니다. 주어진 데이터에 포함된 방언 단어를 참조하여 표준어 문장을 반환하세요.
 
예시:
입력: '누게 있수과' 
출력: '누가 있습니까?'
 
입력: '겅 하니까이'
출력:  '그러니까'
입력: '예초하다가이 뱀 나온거야' 
출력: '예초하다가 뱀이 나온 거야'
제출된 문장에서 제주도 방언 단어를 찾아 주어진 데이터와 대조한 후, 가장 적절한 표준어 번역을 제공하세요.
"""       
        
    })

    # 이전 메시지 포함.
    for history in histories:
        for text in history:
            message_list.append({
                "role": "assistant",
                "content": text
                
            })

    # 유저 메시지.
    message_list.append({
        "role": "user",
        "content": prompt
    })            
    
    payload = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "semantic_configuration": "jeju-dialect-sementic-delete",
                "query_type": "semantic",
                "queryLanguage": "ko-kr",
                "strictness": 2,
                "top_n_documents": 10,
                "key": ai_search_key,
                "indexName": ai_search_index
            }
        }
        ],
    }  
    
    response = requests.post(endpoint, headers=headers, json=payload)
    print(response.status_code)
    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
        content = response_json["choices"][0]["message"]["content"]

        citations = response_json["choices"][0]["message"].get("context", {}).get("citations", [])
        formatted_citation_list = []
        
        if citations:
            for i, citation in enumerate(citations, start=1):
                formatted_citation = f"<details><summary>참조 {i}</summary><ul>{citation['content']}</ul></details>"
                formatted_citation_list.append(formatted_citation)

        return content, "".join(formatted_citation_list)
    else:
        return f"{response.status_code}, {response.text}", ""

def request_stt_fast(file_path):
    endpoint = "https://eastus.api.cognitive.microsoft.com/speechtotext/transcriptions:transcribe?api-version=2024-11-15"
    headers = {
        "Ocp-Apim-Subscription-Key": "EXdCQatIT6OYg5RRAwYr7A6JTvRyDfYzJ8VS6HUMIUpKeB8wcKOqJQQJ99BAACYeBjFXJ3w3AAAYACOGWEOf",
    }

    with open(file_path, "rb") as audio :
        audio_data = audio.read()

    json = {
        "definition": '{"locales":["ko-KR"], "profanityFilterMode":"Masked", "channels":[0,1]}'
    }

    files = {
        "audio" : audio_data
    }

    response = requests.post(endpoint, headers=headers,files=files, json=json)

    print("Fast", response.status_code, response.elapsed.total_seconds())
    if response.status_code == 200:
        response_json = response.json()
        response_text = response_json['combinedPhrases'][0]['text']
        print(response_text)
        return response_text
    else:
        return ''

def click_send(prompt, histories):
    response_text, citation_html = request_gpt(prompt=prompt, histories=histories)
    histories.append((prompt, response_text))
    return histories, citation_html, ""

def change_audio(file_path):
    if file_path:
        response_text = request_stt_fast(file_path=file_path)
        return response_text
    else:
        return ''

with gr.Blocks() as demo:
    
    with gr.Tab(label="🗣️🔄제주방언 번역")  as tab2:
        gr.Markdown("🏝️제주방언 번역")

        # 음성 입력 및 결과 출력
        with gr.Column():
            input_mic = gr.Audio(
                label="🎤 마이크 입력",
                sources="microphone",
                type="filepath",
                interactive=True, waveform_options=gr.WaveformOptions(
                    waveform_color="#D3B1C2",
                    waveform_progress_color="#C197D@",
                    skip_length=2,
                    show_controls=False)
            )



        # 프롬프트 입력과 결과
        gr.Markdown("### 🗣️🔄 번역 내용")
        chatbot_gpt = gr.Chatbot(
            label="🗣️🔄 번역 내용", 
            height=200
        )
        citation = gr.HTML(label="🔗 참조 링크")

        input_openai_textbox = gr.Textbox(
                label="📝 텍스트", 
                interactive=True, 
            )
        send_button = gr.Button("전송")
        
        input_mic.change(
            fn=change_audio, 
            inputs=[input_mic], 
            outputs=[input_openai_textbox]
        )
        send_button.click(
            fn=click_send,
            inputs=[input_openai_textbox, chatbot_gpt],
            outputs=[chatbot_gpt, citation, input_openai_textbox]
        )
        input_openai_textbox.submit(
            fn=click_send,
            inputs=[input_openai_textbox, chatbot_gpt],
            outputs=[chatbot_gpt, citation, input_openai_textbox]
        )

demo.launch()

d:\utility\Python\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
